# Settings

In [ ]:
is_plot = True
is_export = True

landmarks_path = '/Users/knpob/Territory/2-Kolmo/4-Dataset/20230715-DynaBreastLite/landmarks/refine_6kmh_braless_18markers_12fps.pkl'
meshes_path = '/Users/knpob/Territory/2-Kolmo/4-Dataset/20230715-DynaBreastLite/meshes/6kmh_braless_26markers/'
test_landmarks_path = '/Users/knpob/Territory/2-Kolmo/4-Dataset/20230715-DynaBreastLite/test/braless_random_landmarks.pkl'

start=0
stride = 1
end=120

export_folder = "../output/animate/"

# Data loading

In [ ]:
from mesh4d import obj3d
from mesh4d.analyse import crave

mesh_ls, texture_ls = obj3d.load_mesh_series(
    folder=meshes_path,
    start=start,
    stride=stride,
    end=end,
)

In [ ]:
from mesh4d import utils

landmarks_12fps = utils.load_pkl_object(landmarks_path)
landmarks_12fps.interp_field()

landmarks = landmarks_12fps.reslice(120)
landmarks.interp_field()

In [ ]:
from mesh4d.analyse.crave import clip_with_contour

contour = landmarks.extract(('marker 0', 'marker 2', 'marker 3', 'marker 14', 'marker 15', 'marker 17'))
mesh_clip_ls = clip_with_contour(mesh_ls, start_time=0, fps=120/stride, contour=contour, clip_bound='xy', margin=30)

In [ ]:
body_ls = obj3d.init_obj_series(
    mesh_ls,
    obj_type=obj3d.Obj3d_Deform
    )

In [ ]:
breast_ls = obj3d.init_obj_series(
    mesh_clip_ls, 
    obj_type=obj3d.Obj3d_Deform
    )

In [ ]:
from mesh4d import obj4d

body4d = obj4d.Obj4d_Deform(
    fps=120 / stride,
    enable_rigid=False,
    enable_nonrigid=False,
)
body4d.add_obj(*body_ls)
body4d.load_markerset('landmarks', landmarks)

In [ ]:
from mesh4d import obj4d

breast4d = obj4d.Obj4d_Deform(
    fps=120 / stride,
    enable_rigid=False,
    enable_nonrigid=False,
)
breast4d.add_obj(*breast_ls)
breast4d.load_markerset('landmarks', landmarks)

# Data visualisation

In [ ]:
if is_export:
    body4d.animate(output_folder=export_folder, filename='body4d')

In [ ]:
if is_plot:
    body4d.show(elements='m', window_size=[2000, 500], zoom_rate=5, skip=12, m_props={'opacity': 0.5})

In [ ]:
if is_plot:
    body4d.show(elements='mk', window_size=[2000, 500], zoom_rate=5, skip=12, m_props={'opacity': 0.5})

In [ ]:
if is_export:
    breast4d.animate(output_folder=export_folder, filename='breast4d')
    breast4d.animate(output_folder=export_folder, filename='breast4d_mesh', elements='m')

In [ ]:
if is_plot:
    import pyvista as pv

    stack_dist = 1000
    zoom_rate = 3.5
    window_size = [2000, 800]
    
    scene = pv.Plotter()
    plot_num = len(body4d.obj_ls)
    skip = 12

    for idx in range(0, plot_num, skip):
        body = body4d.obj_ls[idx]
        breast = breast4d.obj_ls[idx]
        width = body.get_width()

        body.add_mesh_to_scene(scene, location=[0, 0, idx * stack_dist / skip], opacity=0.1)
        breast.add_mesh_to_scene(scene, location=[0, 0, idx * stack_dist / skip])
        breast.add_kps_to_scene(scene, location=[0, 0, idx * stack_dist / skip], radius=0.02*width)
        
    scene.camera_position = 'zy'
    scene.camera.azimuth = 45
    scene.camera.zoom(zoom_rate)
    scene.window_size = window_size
    scene.enable_parallel_projection()
    scene.show()